Петухова Ксения БКЛ182

In [206]:
import pandas as pd
import RAKE
import nltk
from pymorphy2 import MorphAnalyzer
from pymorphy2.tokenizers import simple_word_tokenize
from nltk.corpus import stopwords
from summa import keywords
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import re
from nltk import word_tokenize, sent_tokenize 
from nltk.util import ngrams
# nltk.download('stopwords')
stop = stopwords.words('russian')
m = MorphAnalyzer()

### Задание 1.

Собранный мною корпус состоит из текстов новостей The Village (https://www.the-village.ru). В качестве ключевых слов были взяты "теги", которые прописаны в конце каждой новости.

In [336]:
corpus = pd.read_csv("the_village.csv", sep=";")

В столбце "Village_keywords" - теги с сайта, в столбце "My_keywords" - ключевые слова, размеченные мной

In [337]:
corpus

,Text,Village_keywords,My_keywords
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и..."
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы"
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow"
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на..."


### Задание 2.

В качестве эталонной разметки возьмем пересечение разметки the village и моей разметки

In [338]:
def intersection():
    gold_kw = []
    village_keywords = list(corpus.Village_keywords)
    my_keywords = list(corpus.My_keywords)
    for i in range(len(my_keywords)): 
        gold_kw_per_text = []
        list_my_keywords = my_keywords[i].split(', ')
        list_v_keywords = village_keywords[i].split(', ')
        for keyword in list_my_keywords:
            if keyword in list_v_keywords:
                gold_kw_per_text.append(keyword)
        gold_kw.append(gold_kw_per_text)
        
    corpus['Gold'] = gold_kw
    return gold_kw

In [339]:
gold_kw = intersection()

На первый взгляд кажется, что разметка не сильно пересекается, но этому у меня есть 2 объяснения:

1) И у меня, и у The Village, ключевых слов достаточно мало;

2) Некоторые теги The Village не встречаются в тексте;

In [340]:
corpus

,Text,Village_keywords,My_keywords,Gold
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]"
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]"
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]"
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]"
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]"
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]"
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы","[аэрофлот, отмена рейсов]"
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow","[значки, сказки, heart of moscow]"
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на...","[сергей собянин, масочный режим, кофемания, ка..."


### Задание 3.

#### RAKE

In [341]:
def normalize_text(text):
    lemmas = []
    for t in simple_word_tokenize(text):
        lemmas.append(
            m.parse(t)[0].normal_form
        )
    return ' '.join(lemmas)

In [342]:
def rake_kw():
    kw_rake = []
    rake = RAKE.Rake(stop)
    for text in list(corpus.Text):
        kw_per_text = []
        kw_list = rake.run(normalize_text(text), maxWords=2, minFrequency=1)
        for l in kw_list:
            kw_per_text.append(l[0])
        kw_rake.append(kw_per_text)
        
    corpus['RAKE'] = kw_rake
    return kw_rake

In [343]:
kw_rake = rake_kw()

In [344]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ..."
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы","[аэрофлот, отмена рейсов]","[возобновление авиасообщение, правительство об..."
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow","[значки, сказки, heart of moscow]","[герой сказка, продажа ещё, третьяковский гале..."
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на...","[сергей собянин, масочный режим, кофемания, ка...","[стать штрафовать, речь идти, средство защита,..."


#### TextRank

In [345]:
def textrank_kw():
    kw_textrank = []
    for text in list(corpus.Text):
        kw_per_text = []
        kw_list = keywords.keywords(normalize_text(text), 
                                    language='russian', additional_stopwords=stop, scores=True)
        for l in kw_list:
            kw_per_text.append(l[0])
        kw_textrank.append(kw_per_text)
        
    corpus['TextRank'] = kw_textrank
    return kw_textrank

In [346]:
kw_textrank = textrank_kw()

In [347]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE,TextRank
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год...","[год, art, галерея, работа, показать, создать,..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х...","[манеж, лабиринт, выставочный проект, дизайн, ..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц...","[вакцина, центр, российский, исследование перв..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем...","[никто, охотка, проект, окно исторический здан..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, владелец трк, подать, ..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов...","[большой, онлайн, мужчина, год, интернет, весь..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ...","[жильё, договор, мочь, должный, банк, кредитны..."
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы","[аэрофлот, отмена рейсов]","[возобновление авиасообщение, правительство об...","[август, рейс, полёт, который, аэрофлот, дата,..."
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow","[значки, сказки, heart of moscow]","[герой сказка, продажа ещё, третьяковский гале...","[русский, сказка, серия значок, точка, бренд, ..."
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на...","[сергей собянин, масочный режим, кофемания, ка...","[стать штрафовать, речь идти, средство защита,...","[елена андреев, москва, собянин, несколько рес..."


#### TFIDF

In [348]:
def tfidf_kw():
    norm_text = [normalize_text(text) for text in list(corpus.Text)]
    kw_tfidf = []
    vectorizer = TfidfVectorizer(use_idf=True, norm='l2', stop_words=stop, ngram_range=(1, 3))
    tfidf = vectorizer.fit_transform(norm_text)
    words = np.array(vectorizer.get_feature_names())
    for vec in tfidf:
        index_sorted = np.argsort(vec.toarray()).ravel()[::-1]
        kw_per_text = words[index_sorted][:10].tolist()
        kw_tfidf.append(kw_per_text)
    
    corpus['TFIDF'] = kw_tfidf
    return kw_tfidf

In [349]:
kw_tfidf = tfidf_kw()

In [350]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE,TextRank,TFIDF
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год...","[год, art, галерея, работа, показать, создать,...","[ярмарка, cosmoscow, галерея, показать, искусс..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х...","[манеж, лабиринт, выставочный проект, дизайн, ...","[кириллический, лабиринт, манеж, проект, выста..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц...","[вакцина, центр, российский, исследование перв...","[вакцина, исследование, москвич, гамалея, учас..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем...","[никто, охотка, проект, окно исторический здан...","[окно, никто, здание, охотка, договор, проект,..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, владелец трк, подать, ...","[левченко, запрет работа, работа, работать, за..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов...","[большой, онлайн, мужчина, год, интернет, весь...","[женщина, мужчина, интернет, онлайн, магазин, ..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ...","[жильё, договор, мочь, должный, банк, кредитны...","[банк, договор, жильё, законопроект, вклад, сб..."
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы","[аэрофлот, отмена рейсов]","[возобновление авиасообщение, правительство об...","[август, рейс, полёт, который, аэрофлот, дата,...","[рейс, билет, полёт, август, аэрофлот, отмена,..."
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow","[значки, сказки, heart of moscow]","[герой сказка, продажа ещё, третьяковский гале...","[русский, сказка, серия значок, точка, бренд, ...","[значок, сказка, серия, of, heart of moscow, o..."
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на...","[сергей собянин, масочный режим, кофемания, ка...","[стать штрафовать, речь идти, средство защита,...","[елена андреев, москва, собянин, несколько рес...","[собянин, нарушение, выявить нарушение, веранд..."


### Задание 4. 

Составляем морфологические/синтаксические шаблоны для ключевых слов и фраз

In [351]:
def extraxt_patterns(gold_kw):
    patterns = []
    for kws in gold_kw:
        for kw in kws:
            splitted_kw = kw.split()
            pos_phrase = []
            for s_kw in splitted_kw:
                if re.search('[a-zA-Z]', s_kw):
                    pos = 'ENG_NAME'
                else:
                    p = m.parse(s_kw)[0]
                    pos = p.tag.POS
                pos_phrase.append(pos)
                
            pos_phrase = ' '.join(pos_phrase)
            if pos_phrase not in patterns:
                patterns.append(pos_phrase)
                
    return patterns

In [352]:
patterns = extraxt_patterns(gold_kw)

In [353]:
patterns

['NOUN', 'ADJF NOUN', 'NOUN NOUN', 'ENG_NAME', 'ENG_NAME ENG_NAME ENG_NAME']

Выделяем соответствующие им подстроки из корпуса

In [354]:
def find_patterns_text(patterns):
    all_extracted = []
    for text in list(corpus.Text):
        extracted = []
        sentences = sent_tokenize(text)
        for sent in sentences:
            tokens = nltk.word_tokenize(sent)
            bigrams = list(ngrams(tokens, 2))
            trigrams = list(ngrams(tokens, 3))
            for pattern in patterns:
                splitted_pattern = pattern.split()
                if len(splitted_pattern) == 1:
                    for token in tokens:
                        if re.search('[a-zA-Z]', token):
                            pos = 'ENG_NAME'
                        else:
                            p = m.parse(token)[0]
                            pos = p.tag.POS

                        if (str(pos) == splitted_pattern[0]) and (token not in extracted):
                            extracted.append(token)
                elif len(splitted_pattern) == 2:
                    for bigram in bigrams:
                        if re.search('[a-zA-Z]', bigram[0][0]):
                            pos1 = 'ENG_NAME'
                        else:
                            p1 = m.parse(bigram[0][0])[0]
                            pos1 = p1.tag.POS
                            
                        if re.search('[a-zA-Z]', bigram[1][0]):
                            pos2 = 'ENG_NAME'
                        else:
                            p2 = m.parse(bigram[1][0])[0]
                            pos2 = p2.tag.POS
                            
                        if (str(pos1) == splitted_pattern[0]) and (str(pos2) == splitted_pattern[1]):
                            match = bigram[0][0] + ' ' + bigram[1][0]
                            if match not in extracted:
                                extracted.append(match)
                else:
                    for trigram in trigrams:
                        if re.search('[a-zA-Z]', trigram[0][0]):
                            pos1 = 'ENG_NAME'
                        else:
                            p1 = m.parse(trigram[0][0])[0]
                            pos1 = p1.tag.POS
                            
                        if re.search('[a-zA-Z]', trigram[1][0]):
                            pos2 = 'ENG_NAME'
                        else:
                            p2 = m.parse(trigram[1][0])[0]
                            pos2 = p2.tag.POS
                        
                        if re.search('[a-zA-Z]', trigram[2][0]):
                            pos3 = 'ENG_NAME'
                        else:
                            p3 = m.parse(trigram[2][0])[0]
                            pos3 = p3.tag.POS
                        
                        if (str(pos1) == splitted_pattern[0]) and (str(pos2) == splitted_pattern[1]) \
                            and (str(pos3) == splitted_pattern[2]):
                            match = trigram[0][0] + ' ' + trigram[1][0] + ' ' + trigram[2][0]
                            if match not in extracted:
                                extracted.append(match)
                                
        all_extracted.append(extracted)
    
    corpus['Appropriate_pattern_string'] = all_extracted
    
    return all_extracted


In [355]:
all_extracted = find_patterns_text(patterns)

In [356]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE,TextRank,TFIDF,Appropriate_pattern_string
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год...","[год, art, галерея, работа, показать, создать,...","[ярмарка, cosmoscow, галерея, показать, искусс...","[сентября, дворе, ярмарка, искусства, Cosmosco..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х...","[манеж, лабиринт, выставочный проект, дизайн, ...","[кириллический, лабиринт, манеж, проект, выста...","[Манеж, проект, знака, алфавиту, Выставка, инс..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц...","[вакцина, центр, российский, исследование перв...","[вакцина, исследование, москвич, гамалея, учас...","[центр, имени, Гамалеи, поддержке, фонда, инве..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем...","[никто, охотка, проект, окно исторический здан...","[окно, никто, здание, охотка, договор, проект,...","[ресторан, Охотка, конце, июня, Маросейке, ули..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, владелец трк, подать, ...","[левченко, запрет работа, работа, работать, за...","[Владельцы, Европолис, суд, Фонтанке, партнер,..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов...","[большой, онлайн, мужчина, год, интернет, весь...","[женщина, мужчина, интернет, онлайн, магазин, ...","[Банк, стандарт, россияне, интернет-магазинах,..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ...","[жильё, договор, мочь, должный, банк, кредитны...","[банк, договор, жильё, законопроект, вклад, сб...","[Госдуму, законопроект, запуске, механизма, си..."
7,Несмотря на возобновление авиасообщения с авгу...,"авиасообщение, авиасообщение с турцией, отмена...","аэрофлот, отмена рейсов, август, закрытые границы","[аэрофлот, отмена рейсов]","[возобновление авиасообщение, правительство об...","[август, рейс, полёт, который, аэрофлот, дата,...","[рейс, билет, полёт, август, аэрофлот, отмена,...","[возобновление, авиасообщения, августа, России..."
8,Бренд московских сувениров Heart of Moscow вып...,"значки, сказки, картины, художники, heart of m...","значки, сказки, heart of moscow","[значки, сказки, heart of moscow]","[герой сказка, продажа ещё, третьяковский гале...","[русский, сказка, серия значок, точка, бренд, ...","[значок, сказка, серия, of, heart of moscow, o...","[Бренд, сувениров, серию, значков, героями, ск..."
9,Мэра Москвы Сергея Собянина не стали штрафоват...,"кофемания, масочный режим, карантин, штраф, се...","собянин, маска, мэр москвы, сергей собянин, на...","[сергей собянин, масочный режим, кофемания, ка...","[стать штрафовать, речь идти, средство защита,...","[елена андреев, москва, собянин, несколько рес...","[собянин, нарушение, выявить наруше

Применяем эти фильтры к спискам ключевых слов

In [357]:
def filter_kw(keywords, patterns):
    all_filtered_kws = []
    for kws in keywords:
        filtered_kws = []
        for kw in kws:
            for pattern in patterns:
                if len(kw.split(' ')) == len(pattern.split(' ')):
                    kw_pos = []
                    for i in range(len(kw.split(' '))):
                        if re.search('[a-zA-Z]', kw.split(' ')[i]):
                            pos = 'ENG_NAME'
                        else:
                            p = m.parse(kw.split(' ')[i])[0]
                            pos = p.tag.POS
                            
                        kw_pos.append(str(pos))
                    
                if ' '.join(kw_pos) == pattern:
                    filtered_kws.append(kw)
                        
        all_filtered_kws.append(filtered_kws)
        
    return all_filtered_kws

In [358]:
filtered_rake = filter_kw(kw_rake, patterns)
corpus['filtered_RAKE'] = filtered_rake

In [359]:
filtered_textrank = filter_kw(kw_textrank, patterns)
corpus['filtered_TextRank'] = filtered_textrank

In [360]:
filtered_tfidf = filter_kw(kw_tfidf, patterns)
corpus['filtered_TFIDF'] = filtered_tfidf

In [361]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE,TextRank,TFIDF,Appropriate_pattern_string,filtered_RAKE,filtered_TextRank,filtered_TFIDF
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год...","[год, art, галерея, работа, показать, создать,...","[ярмарка, cosmoscow, галерея, показать, искусс...","[сентября, дворе, ярмарка, искусства, Cosmosco...","[a-s-t-r-agallery, импровизированный галерея, ...","[год, art, галерея, работа, сад, конкурс, янде...","[ярмарка, cosmoscow, галерея, искусство, совре..."
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х...","[манеж, лабиринт, выставочный проект, дизайн, ...","[кириллический, лабиринт, манеж, проект, выста...","[Манеж, проект, знака, алфавиту, Выставка, инс...","[юрий гордон, митя харшак, качество куратор, с...","[манеж, лабиринт, выставочный проект, выставоч...","[лабиринт, манеж, проект, арт объект, буква, а..."
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц...","[вакцина, центр, российский, исследование перв...","[вакцина, исследование, москвич, гамалея, учас...","[центр, имени, Гамалеи, поддержке, фонда, инве...","[свой блог, свой блог, сайт мэрия, медицинский...","[вакцина, центр, участник, человек, москвич, м...","[вакцина, исследование, москвич, гамалея, учас..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем...","[никто, охотка, проект, окно исторический здан...","[окно, никто, здание, охотка, договор, проект,...","[ресторан, Охотка, конце, июня, Маросейке, ули...","[конец июнь, прошлый неделя, местный активист,...","[охотка, проект, окно исторический здание, дог...","[окно, здание, охотка, договор, проект, догово..."
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, владелец трк, подать, ...","[левченко, запрет работа, работа, работать, за...","[Владельцы, Европолис, суд, Фонтанке, партнер,...","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, ресторан, ресторанный ...","[левченко, запрет работа, запрет работа, работ..."
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов...","[большой, онлайн, мужчина, год, интернет, весь...","[женщина, мужчина, интернет, онлайн, магазин, ...","[Банк, стандарт, россияне, интернет-магазинах,...","[доля интернет-торговля, интернет-магазин, онл...","[мужчина, год, интернет, рост, рубль, покупка,...","[женщина, мужчина, интернет, магазин, триллион..."
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ...","[жильё, договор, мочь, должный, банк, кредитны...","[банк, договор, жильё, законопроект, вклад, сб...","[Госдуму, законопроект, запуске, механизма, си...","[покупка жильё, срок договор, приобретение жил...","[жильё, договор, мочь, банк, автор, вклад, про...","[банк, договор, жильё, законопроек

### Задание 5. 

Оцениваем точность, полноту, F-меру выбранных методов относительно эталона: с учётом морфосинтаксических шаблонов и без них

In [362]:
norm_gold = [[normalize_text(kw) for kw in kws] for kws in list(corpus.Gold)]

In [363]:
corpus['normalized_gold'] = norm_gold

In [364]:
corpus

,Text,Village_keywords,My_keywords,Gold,RAKE,TextRank,TFIDF,Appropriate_pattern_string,filtered_RAKE,filtered_TextRank,filtered_TFIDF,normalized_gold
0,С 11 по 13 сентября в Гостином дворе пройдет у...,"современное искусство, выставка, ярмарка, сosm...","ярмарка, современное искусство, cosmoscow, сен...","[ярмарка, современное искусство, гостиный двор]","[a-s-t-r-agallery, зато онлайн-формат, это год...","[год, art, галерея, работа, показать, создать,...","[ярмарка, cosmoscow, галерея, показать, искусс...","[сентября, дворе, ярмарка, искусства, Cosmosco...","[a-s-t-r-agallery, импровизированный галерея, ...","[год, art, галерея, работа, сад, конкурс, янде...","[ярмарка, cosmoscow, галерея, искусство, совре...","[ярмарка, современный искусство, гостиный двор]"
1,ЦВЗ «Манеж» запустил выставочный проект «33 зн...,"выставки, уличное искусство, уличные выставки,...","манеж, выставочный проект, 33 знака, кириллица...","[манеж, кириллица, алфавит]","[александр шишкин-хокусать, александр шишкин-х...","[манеж, лабиринт, выставочный проект, дизайн, ...","[кириллический, лабиринт, манеж, проект, выста...","[Манеж, проект, знака, алфавиту, Выставка, инс...","[юрий гордон, митя харшак, качество куратор, с...","[манеж, лабиринт, выставочный проект, выставоч...","[лабиринт, манеж, проект, арт объект, буква, а...","[манеж, кириллица, алфавит]"
2,Исследовательский центр имени Гамалеи при подд...,"коронавирус, вакцинация, вакцина, covid-19, се...","москва, вакцина, коронавирус, сергей собянин, ...","[вакцина, коронавирус, сергей собянин, вакцина...","[свой блог, оставить заявка, сайт мэрия, медиц...","[вакцина, центр, российский, исследование перв...","[вакцина, исследование, москвич, гамалея, учас...","[центр, имени, Гамалеи, поддержке, фонда, инве...","[свой блог, свой блог, сайт мэрия, медицинский...","[вакцина, центр, участник, человек, москвич, м...","[вакцина, исследование, москвич, гамалея, учас...","[вакцина, коронавирус, сергей собянин, вакцина..."
3,Рыбный ресторан «Охотка» открылся в конце июня...,"архитектура, сохранение исторического центра, ...","охотка, маросейка, уничтожение окон, историчес...","[охотка, маросейка]","[конец июнь, прошлый неделя, появиться проблем...","[никто, охотка, проект, окно исторический здан...","[окно, никто, здание, охотка, договор, проект,...","[ресторан, Охотка, конце, июня, Маросейке, ули...","[конец июнь, прошлый неделя, местный активист,...","[охотка, проект, окно исторический здание, дог...","[окно, здание, охотка, договор, проект, догово...","[охотка, маросейка]"
4,Владельцы ТРК «Европолис» — Fort Group — подал...,"бизнес, суд, чиновники, ограничения, торговый ...","трк Европолис, fort group, суд, смольный, рабо...","[суд, ограничения]","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, владелец трк, подать, ...","[левченко, запрет работа, работа, работать, за...","[Владельцы, Европолис, суд, Фонтанке, партнер,...","[ресторанный дворик, торговый центр, развлекат...","[европолис, fort group, ресторан, ресторанный ...","[левченко, запрет работа, запрет работа, работ...","[суд, ограничение]"
5,"Банк «Русский стандарт» изучил, что россияне п...","онлайн-шопинг, торговля, ритейл, деньги, интер...","мужчины, онлайн-магазины, женщины, товары для ...","[исследование, деньги]","[измениться онлайн-рынок, доля интернет-торгов...","[большой, онлайн, мужчина, год, интернет, весь...","[женщина, мужчина, интернет, онлайн, магазин, ...","[Банк, стандарт, россияне, интернет-магазинах,...","[доля интернет-торговля, интернет-магазин, онл...","[мужчина, год, интернет, рост, рубль, покупка,...","[женщина, мужчина, интернет, магазин, триллион...","[исследование, деньга]"
6,В Госдуму внесли законопроект о запуске механи...,"вклады, покупка жилья, кредиты, ипотека, закон...","вклад, покупка жилья, госдума, законопроект, и...","[покупка жилья, госдума, законопроект]","[помочь гражданин, покупка жильё, накапливать ...","[жильё, договор, мочь, должный, банк, кредитны...","[банк, договор, жильё, зак

In [365]:
def metrics(keywords, gold_kw):
    all_precision = 0
    all_recall = 0
    all_f1 = 0
    for i in range(len(keywords)):
        tp = 0
        fp = 0 
        fn = 0
        for kw in keywords[i]:
            if kw in gold_kw[i]:
                tp += 1
            else:
                fp += 1
    
        for kw in gold_kw[i]:
            if kw not in keywords[i]:
                fn += 1

        precision = tp / (tp + fp)
        all_precision += precision
        recall = tp / (tp + fn)
        all_recall += recall
        if (precision == 0) or (recall == 0):
            f1 = 0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)
        
        all_f1 += f1
        
    return all_precision/len(keywords), all_recall/len(keywords), all_f1/len(keywords)

In [367]:
print("RAKE")
precision, recall, f1 = metrics(kw_rake, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

RAKE
precision:  0.02678250108397167
recall:  0.30166666666666664
f1:  0.04802481634728286


In [368]:
print("TextRank")
precision, recall, f1 = metrics(kw_textrank, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

TextRank
precision:  0.026185721294416948
recall:  0.22666666666666666
f1:  0.045537599655246716


In [369]:
print("TFIDF")
precision, recall, f1 = metrics(kw_tfidf, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

TFIDF
precision:  0.13
recall:  0.44333333333333336
f1:  0.19769230769230767


In [371]:
print("Filtered RAKE")
precision, recall, f1 = metrics(filtered_rake, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

Filtered RAKE
precision:  0.05177225768356122
recall:  0.30166666666666664
f1:  0.08481476594684143


In [372]:
print("Filtered TextRank")
precision, recall, f1 = metrics(filtered_textrank, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

Filtered TextRank
precision:  0.039186335403726714
recall:  0.22666666666666666
f1:  0.06541624639450726


In [373]:
print("Filtered TextRank")
precision, recall, f1 = metrics(filtered_tfidf, norm_gold)
print("precision: ", precision)
print("recall: ", recall)
print("f1: ", f1)

Filtered TextRank
precision:  0.17150793650793653
recall:  0.45166666666666666
f1:  0.24398379398379402


### Задание 6. 

Ошибки автоматического выделения ключевых слов:
* Среди нефильтрованных автоматически выделенных слов встречаются глаголы
* Среди нефильтрованных автоматически выделенных слов встречаются числа. Когда встречаются число + существительное, то это нормально, под такой шаблон часто попадают даты, но когда встречается просто число, то это уже не информативно
* Некоторые знаки препинания (напр. '»'), не выбрасываются и входят в список стоп слов
* Несмотря на то, что мы убрали стоп-слова с помощью нлтк, среди автоматически выделенных слов все равно встречаются некоторые предлоги и союзы
* В текстранке часто встречаются прилагательные по отдельности, что тоже не несет особого смысла

 Методы решения этих проблем:
 * Проблему с глаголами, числами и прилагательными как раз решают морфосинтаксические шаблоны
 * Проблему со знаками препинания можно решить пост-очисткой от пунктуации
 * Проблему со стоп-словами можно решить ручным расширением списка стоп-слов